## ここから

In [29]:
import pandas as pd
import numpy as np
import random

# ---------------------------------------------------------
# 設定
# ---------------------------------------------------------
NUM_PLAYERS = 12         # 12人の投手を作成
PITCHES_PER_PLAYER = 300 # 1人あたり300球
YEAR = 2025

print("📂 データ生成を開始します...")

# 投手のタイプ定義（分析結果に差が出るように）
types = [
    {'name': '本格派', 'fast': 0.5, 'breaking': ['フォーク', 'スライダー'], 'k_rate': 0.3},
    {'name': '軟投派', 'fast': 0.3, 'breaking': ['カーブ', 'シンカー', 'シュート'], 'k_rate': 0.15},
    {'name': 'ドクターK', 'fast': 0.4, 'breaking': ['スライダー', 'フォーク'], 'k_rate': 0.45}
]

# ---------------------------------------------------------
# 1. stats_pit.csv の作成
# ---------------------------------------------------------
stats_data = []
pitcher_meta = [] # play_info生成用に情報を保存しておくリスト

for i in range(NUM_PLAYERS):
    # IDと名前
    p_id = 10001 + i
    p_type = random.choice(types)
    p_name = f"投手{chr(65+i)}_{p_type['name']}" # 投手A_本格派...
    hand = random.choice(['R', 'L']) # 利き腕
    
    # 投球回(ip)と奪三振(so)の計算
    ip = np.random.randint(100, 180)
    so = int(ip * p_type['k_rate'] * np.random.uniform(0.9, 1.1) * 3)
    
    # メタ情報を保存（次のループで使う）
    pitcher_meta.append({
        'id': p_id, 
        'name': p_name, 
        'hand': hand, 
        'type': p_type
    })
    
    # 指定されたカラム通りに作成
    stats_data.append({
        'season_year': YEAR,
        'player_id': p_id,
        'player_name': p_name,
        'g': 25,                       # 登板数
        'gs': 25,                      # 先発数
        'w': np.random.randint(5, 15), # 勝利
        'l': np.random.randint(3, 10), # 敗戦
        'sv': 0,                       # セーブ
        'hld': 0,                      # ホールド
        'ip': ip,                      # 投球回
        'er': np.random.randint(30, 60), # 自責点
        'so': so,                      # 奪三振
        'np': ip * 15,                 # 総投球数
        'bf': ip * 4                   # 対戦打者数
    })

df_stats = pd.DataFrame(stats_data)
# 指定された並び順に整える
cols_stats = ['season_year', 'player_id', 'player_name', 'g', 'gs', 'w', 'l', 'sv', 'hld', 'ip', 'er', 'so', 'np', 'bf']
df_stats = df_stats[cols_stats]

df_stats.to_csv('test_stats_pit.csv', index=False)
print(f"✅ 'test_stats_pit.csv' を作成しました（12人分）")

# ---------------------------------------------------------
# 2. play_info.csv の作成
# ---------------------------------------------------------
play_data = []

for p in pitcher_meta:
    # その投手の特徴を取り出す
    p_balls = ['ストレート'] + p['type']['breaking']
    
    for _ in range(PITCHES_PER_PLAYER):
        # 状況のランダム生成
        ball = random.choice(p_balls)
        speed = np.random.randint(130, 160)
        
        # 空振り判定ロジック（決め球は空振りしやすい）
        is_money_pitch = (p['type']['name'] == 'ドクターK' and ball == p['type']['breaking'][0])
        prob_swing = 0.4 if is_money_pitch else (0.05 if ball == 'ストレート' else 0.15)
        
        is_swinging = 1 if random.random() < prob_swing else 0
        is_called = 1 if (is_swinging == 0 and random.random() < 0.3) else 0 # 空振りじゃないなら30%で見逃しS
        
        play_data.append({
            'game_id': f"game_{random.randint(1, 100)}",
            'game_year': YEAR,
            'pitcher_id': p['id'],
            'pitcher_name': p['name'],
            'pitcher_handedness': p['hand'],
            'batter_id': random.randint(20000, 29999),
            'batter_name': f"打者{random.randint(1, 100)}",
            'batter_handedness': random.choice(['R', 'L']),
            'inning': random.randint(1, 9),
            'pre_ball': random.randint(0, 3),
            'pre_strike': random.randint(0, 2),
            'pre_out': random.randint(0, 2),
            'is_risp': random.choice([0, 1]), # 得点圏か
            'pitch_type_name': ball,
            'pitch_speed': speed,
            'pitch_location_x': round(np.random.uniform(-1.0, 1.0), 2),
            'pitch_location_y': round(np.random.uniform(0.0, 1.5), 2),
            'is_swinging_strike': is_swinging,
            'is_called_strike': is_called
        })

df_play = pd.DataFrame(play_data)
# 指定された並び順に整える
cols_play = [
    'game_id', 'game_year', 'pitcher_id', 'pitcher_name', 'pitcher_handedness', 
    'batter_id', 'batter_name', 'batter_handedness', 'inning', 'pre_ball', 
    'pre_strike', 'pre_out', 'is_risp', 'pitch_type_name', 'pitch_speed', 
    'pitch_location_x', 'pitch_location_y', 'is_swinging_strike', 'is_called_strike'
]
df_play = df_play[cols_play]

df_play.to_csv('test_play_info.csv', index=False)
print(f"✅ 'test_play_info.csv' を作成しました（{len(df_play)}球）")
print("🎉 これでファイルの中身（カラム名）は完璧です！")

📂 データ生成を開始します...
✅ 'test_stats_pit.csv' を作成しました（12人分）
✅ 'test_play_info.csv' を作成しました（3600球）
🎉 これでファイルの中身（カラム名）は完璧です！


In [36]:
import pandas as pd
import numpy as np
import random

# ---------------------------------------------------------
# 設定：12人で5年分つくる
# ---------------------------------------------------------
NUM_PLAYERS = 12
PITCHES_PER_YEAR = 100 # 1年あたり100球（処理を軽くするため）
YEARS = range(2021, 2026) # 2021, 22, 23, 24, 25

print("📂 5年分のデータ生成を開始します...")

# 投手のベースタイプ
types = [
    {'name': '本格派', 'fast': 0.5, 'breaking': ['フォーク', 'スライダー'], 'k_rate': 0.3},
    {'name': '軟投派', 'fast': 0.3, 'breaking': ['カーブ', 'シンカー', 'シュート'], 'k_rate': 0.15},
    {'name': 'ドクターK', 'fast': 0.4, 'breaking': ['スライダー', 'フォーク'], 'k_rate': 0.45}
]

play_data = []
stats_data = []

# 12人の投手を作成
pitchers = []
for i in range(NUM_PLAYERS):
    p_type = random.choice(types)
    p_name = f"投手{chr(65+i)}_{p_type['name']}"
    pitchers.append({'name': p_name, 'type': p_type})

# 年ごとにデータを生成
for year in YEARS:
    for p in pitchers:
        # その年の成績（ランダム）
        stats_data.append({
            'season_year': year,
            'player_name': p['name'],
            'so': np.random.randint(50, 200), # 奪三振
            'ip': np.random.randint(50, 150),
            'era': np.random.uniform(2.0, 5.0)
        })
        
        # その年の投球データ
        p_type = p['type']['name']
        balls = ['ストレート'] + p['type']['breaking']
        
        # 年によって少しスタイルを変える（成長のシミュレーション）
        # 偶数年はサプライズ（裏かき）が増える設定
        surprise_tendency = 0.4 if year % 2 == 0 else 0.1
        
        for _ in range(PITCHES_PER_YEAR):
            ball = random.choice(balls)
            
            # 確率設定
            is_money_pitch = (ball == p['type']['breaking'][0])
            if is_money_pitch:
                prob_swing = 0.3 + (0.1 if random.random() < surprise_tendency else 0)
            else:
                prob_swing = 0.1
                
            is_swing = 1 if random.random() < prob_swing else 0
            is_called = 1 if (is_swing == 0 and random.random() < 0.3) else 0
            
            play_data.append({
                'game_year': year,           # ★ここが重要：年度
                'pitcher_name': p['name'],
                'pitch_type_name': ball,
                'pre_ball': random.randint(0, 3),
                'pre_strike': random.randint(0, 2),
                'is_swinging_strike': is_swing,
                'is_called_strike': is_called
            })

# 保存
df_stats = pd.DataFrame(stats_data)
df_play = pd.DataFrame(play_data)

df_stats.to_csv('test_stats_pit.csv', index=False)
df_play.to_csv('test_play_info.csv', index=False)

print(f"✅ 5年分のデータ作成完了！ (全{len(df_play)}球)")

📂 5年分のデータ生成を開始します...
✅ 5年分のデータ作成完了！ (全6000球)


In [30]:
import pandas as pd

# 1. データの読み込み
df_stats = pd.read_csv('test_stats_pit.csv')
df_play = pd.read_csv('test_play_info.csv')

# 防御率計算
df_stats['era'] = df_stats.apply(
    lambda row: (row['er'] * 9) / row['ip'] if row['ip'] > 0 else 0.00, axis=1
).round(2)

# ---------------------------------------------------------
# ★ここがポイント：グラフ用の「まとめリスト」を作る
# ---------------------------------------------------------
viz_data_list = []

# 奪三振が多い順に処理
target_pitchers = df_stats.sort_values(by='so', ascending=False)

for index, row in target_pitchers.iterrows():
    p_name = row['player_name']
    
    # その投手の全投球データ
    df_target = df_play[df_play['pitcher_name'] == p_name]
    
    if len(df_target) == 0:
        continue

    # --- A. 決め球データの抽出 ---
    # 球種ごとの成績を計算
    ball_stats = df_target.groupby('pitch_type_name').agg(
        count=('pitch_type_name', 'count'),
        whiff=('is_swinging_strike', 'sum')
    ).reset_index()
    
    ball_stats['whiff_rate'] = ball_stats['whiff'] / ball_stats['count']
    
    # 最も空振り率が高い球（10球以上投げているもの）を探す
    valid_balls = ball_stats[ball_stats['count'] >= 10]
    
    if len(valid_balls) > 0:
        # 空振り率No.1の球
        best_ball = valid_balls.sort_values('whiff_rate', ascending=False).iloc[0]
        best_ball_name = best_ball['pitch_type_name']
        best_ball_rate = best_ball['whiff_rate']
    else:
        best_ball_name = "データ不足"
        best_ball_rate = 0

    # --- B. 1行のデータにまとめる ---
    viz_data_list.append({
        '投手名': p_name,
        '防御率': row['era'],
        '奪三振数': row['so'],
        '投球回': row['ip'],
        '決め球名': best_ball_name,       # 色分けに使えそう
        '決め球空振り率': round(best_ball_rate, 3), # 小数点見やすく
        '総投球数': len(df_target)        # バブルチャートのサイズに使えそう
    })

# ---------------------------------------------------------
# データフレーム化 & CSV保存
# ---------------------------------------------------------
df_viz = pd.DataFrame(viz_data_list)

print("✅ グラフ用のデータセット(df_viz)が完成しました！")
display(df_viz)

# ★ここが追加部分！
csv_filename = 'pitcher_summary_12players.csv'
df_viz.to_csv(csv_filename, index=False, encoding='utf-8-sig') # 文字化け防止付き

print(f"💾 '{csv_filename}' という名前で保存しました！")
print("   → 散布図やバブルチャートを作る時はこのファイルを渡してください！")

✅ グラフ用のデータセット(df_viz)が完成しました！


,投手名,防御率,奪三振数,投球回,決め球名,決め球空振り率,総投球数
0,投手K_ドクターK,1.84,214,161,スライダー,0.369,300
1,投手B_ドクターK,1.69,213,170,スライダー,0.378,300
2,投手C_本格派,2.13,152,156,スライダー,0.235,300
3,投手G_ドクターK,4.28,140,103,スライダー,0.370,300
4,投手D_本格派,2.31,137,144,スライダー,0.173,300
5,投手J_本格派,3.28,137,148,スライダー,0.216,300
6,投手F_本格派,2.08,127,143,スライダー,0.186,300
7,投手E_本格派,3.43,102,118,フォーク,0.205,300
8,投手I_本格派,3.54,94,112,フォーク,0.174,300
9,投手L_本格派,3.89,93,104,フォーク,0.149,300


💾 'pitcher_summary_12players.csv' という名前で保存しました！
   → 散布図やバブルチャートを作る時はこのファイルを渡してください！


In [31]:
import pandas as pd

# ---------------------------------------------------------
# 1. データの読み込み
# ---------------------------------------------------------
df_stats = pd.read_csv('test_stats_pit.csv')
df_play = pd.read_csv('test_play_info.csv')

# 奪三振が多い順に並べる（グラフにする時に強い順だと見やすいので）
target_pitchers = df_stats.sort_values(by='so', ascending=False)['player_name'].tolist()

print("📊 配球データ表を作成中...")

# ---------------------------------------------------------
# 2. 集計処理
# ---------------------------------------------------------
data_list = []

for p_name in target_pitchers:
    # その投手の全データ
    df_target = df_play[df_play['pitcher_name'] == p_name]
    
    # 左右別に集計
    for hand in ['R', 'L']:
        label = "右打者" if hand == 'R' else "左打者"
        
        # その利き腕に対するデータ
        df_hand = df_target[df_target['batter_handedness'] == hand]
        
        if len(df_hand) == 0:
            continue
            
        # 球種ごとの数をカウント
        # normalize=True にすると、個数ではなく「割合（0.0〜1.0）」を出してくれます
        counts = df_hand['pitch_type_name'].value_counts(normalize=True)
        
        # データをリストに追加
        for ball, rate in counts.items():
            data_list.append({
                '投手名': p_name,
                '対戦打者': label,  # 右打者 or 左打者
                '球種': ball,
                '投球割合': round(rate * 100, 1) # ％表記にするため100掛けて四捨五入（例: 35.2）
            })

# リストをデータフレーム（表）に変換
df_distribution = pd.DataFrame(data_list)

# ---------------------------------------------------------
# 3. 見やすい表（クロス集計表）に変換
# ---------------------------------------------------------
# そのままだと縦長すぎるので、横に球種が並ぶ形にします
# これが一番「グラフ担当」が喜びます！
df_export = df_distribution.pivot_table(
    index=['投手名', '対戦打者'], 
    columns='球種', 
    values='投球割合'
).fillna(0) # 投げていない球種は0を入れる

# ---------------------------------------------------------
# 4. 確認と保存
# ---------------------------------------------------------
print("\n✅ グラフ担当用のデータが完成しました！")
display(df_export)

# CSVファイルとして保存（これを担当の子に渡してあげてください）
csv_filename = 'pitch_distribution_12players.csv'
df_export.to_csv(csv_filename, encoding='utf-8-sig') # 文字化けしないようにsig付き
print(f"\n💾 '{csv_filename}' という名前で保存しました！")

📊 配球データ表を作成中...

✅ グラフ担当用のデータが完成しました！


球種               カーブ  シュート  シンカー  ストレート  スライダー  フォーク
投手名       対戦打者                                      
投手A_軟投派   右打者   21.7  24.2  26.8   27.4    0.0   0.0
          左打者   30.8  17.5  18.9   32.9    0.0   0.0
投手B_ドクターK 右打者    0.0   0.0   0.0   37.5   27.6  34.9
          左打者    0.0   0.0   0.0   33.1   32.4  34.5
投手C_本格派   右打者    0.0   0.0   0.0   38.6   28.8  32.7
          左打者    0.0   0.0   0.0   36.7   25.2  38.1
投手D_本格派   右打者    0.0   0.0   0.0   32.5   36.7  30.7
          左打者    0.0   0.0   0.0   40.3   32.1  27.6
投手E_本格派   右打者    0.0   0.0   0.0   32.1   37.7  30.2
          左打者    0.0   0.0   0.0   34.8   36.9  28.4
投手F_本格派   右打者    0.0   0.0   0.0   31.1   34.1  34.8
          左打者    0.0   0.0   0.0   29.2   33.9  36.9
投手G_ドクターK 右打者    0.0   0.0   0.0   33.1   33.8  33.1
          左打者    0.0   0.0   0.0   36.1   32.9  31.0
投手H_軟投派   右打者   29.4  22.8  25.7   22.1    0.0   0.0
          左打者   23.2  29.3  26.8   20.7    0.0   0.0
投手I_本格派   右打者    0.0   0.0   0.0   33.3   32.7  34.0
          左打者    0.0   0.0   0.0   29.7   31.2  39.1
投手J_本格派   右打者    0.0   0.0   0.0   34.9   28.3  36.8
          左打者    0.0   0.0   0.0   37.8   30.4  31.8
投手K_ドクターK 右打者    0.0   0.0   0.0   35.0   38.5  26.6
          左打者    0.0   0.0   0.0   34.4   35.7  29.9
投手L_本格派   右打者    0.0   0.0   0.0   39.2   31.1  29.7
          左打者    0.0   0.0   0.0   29.6   37.5  32.9


💾 'pitch_distribution_12players.csv' という名前で保存しました！


In [ ]:
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 1. データの読み込み
# ---------------------------------------------------------
df_play = pd.read_csv('test_play_info.csv')
df_stats = pd.read_csv('test_stats_pit.csv')

# 分析対象の投手（奪三振TOP12）
target_pitchers = df_stats.sort_values(by='so', ascending=False)['player_name'].tolist()

print("🧠 サプライズ指数の計算を開始します...")

# ---------------------------------------------------------
# 2. 「世間の常識（セオリー）」を定義する
# ---------------------------------------------------------
# カウントの状態を表す列を作る
df_play['count_state'] = 'B' + df_play['pre_ball'].astype(str) + '-' + 'S' + df_play['pre_strike'].astype(str)

# リーグ全体での「カウント別・球種選択確率」を計算
league_tendency = df_play.groupby(['count_state', 'pitch_type_name']).size().reset_index(name='count')
count_totals = df_play.groupby('count_state').size().reset_index(name='total')
league_tendency = pd.merge(league_tendency, count_totals, on='count_state')
league_tendency['theory_prob'] = league_tendency['count'] / league_tendency['total']

# 確率辞書を作る
theory_map = dict(zip(zip(league_tendency['count_state'], league_tendency['pitch_type_name']), league_tendency['theory_prob']))

# ---------------------------------------------------------
# 3. 各プレーの「サプライズ度」を計算
# ---------------------------------------------------------
def calculate_surprise(row):
    key = (row['count_state'], row['pitch_type_name'])
    prob = theory_map.get(key, 0)
    return 1.0 - prob # 確率が低いほどサプライズ度が上がる

df_play['surprise_score'] = df_play.apply(calculate_surprise, axis=1)

# ---------------------------------------------------------
# 4. サプライズの性質を分類（攻め vs 無頓着）
# ---------------------------------------------------------
# 成功フラグ（空振り or 見逃しストライクなら 1）
df_play['is_success'] = ((df_play['is_swinging_strike'] == 1) | (df_play['is_called_strike'] == 1)).astype(int)

# 2種類のサプライズを計算
df_play['strategic_score'] = df_play['surprise_score'] * df_play['is_success']       # 攻め（成功）
df_play['random_score'] = df_play['surprise_score'] * (1 - df_play['is_success'])   # 無頓着（失敗）

# ---------------------------------------------------------
# 5. 投手ごとに集計してリスト化
# ---------------------------------------------------------
surprise_analysis = []

for p_name in target_pitchers:
    df_target = df_play[df_play['pitcher_name'] == p_name]
    
    if len(df_target) == 0:
        continue
        
    # 平均値を算出
    avg_surprise = df_target['surprise_score'].mean()
    avg_strategic = df_target['strategic_score'].mean()
    avg_random = df_target['random_score'].mean()
    
    surprise_analysis.append({
        '投手名': p_name,
        'サプライズ指数_総合': round(avg_surprise, 4),
        '攻めるサプライズ': round(avg_strategic, 4),   # 縦軸用
        '無頓着サプライズ': round(avg_random, 4)       # 横軸用
    })

# データフレーム化
df_surprise = pd.DataFrame(surprise_analysis)

# ---------------------------------------------------------
# 6. CSVファイルとして保存
# ---------------------------------------------------------
csv_filename = 'pitcher_surprise_analysis.csv'
df_surprise.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print("✅ 計算完了！")
display(df_surprise)
print(f"💾 '{csv_filename}' を保存しました。グラフ担当の方に渡してください！")

🧠 サプライズ指数の計算を開始します...
✅ 計算完了！


,投手名,サプライズ指数_総合,攻めるサプライズ,無頓着サプライズ
0,投手K_ドクターK,0.7060,0.3232,0.3828
1,投手B_ドクターK,0.7058,0.2925,0.4133
2,投手C_本格派,0.7036,0.2709,0.4326
3,投手G_ドクターK,0.7073,0.2725,0.4348
4,投手D_本格派,0.7056,0.2347,0.4709
5,投手J_本格派,0.7050,0.2531,0.4519
6,投手F_本格派,0.7099,0.2788,0.4311
7,投手E_本格派,0.7044,0.2738,0.4306
8,投手I_本格派,0.7075,0.2752,0.4323
9,投手L_本格派,0.7063,0.2743,0.4320


💾 'pitcher_surprise_analysis.csv' を保存しました。グラフ担当の方に渡してください！


In [37]:
import pandas as pd

# 1. 読み込み
df_play = pd.read_csv('test_play_info.csv')

print("🧠 年度別サプライズ指数の計算を開始します...")

# ---------------------------------------------------------
# 2. セオリー定義（全年度まとめて基準を作る）
# ---------------------------------------------------------
df_play['count_state'] = 'B' + df_play['pre_ball'].astype(str) + '-' + 'S' + df_play['pre_strike'].astype(str)

league_tendency = df_play.groupby(['count_state', 'pitch_type_name']).size().reset_index(name='count')
count_totals = df_play.groupby('count_state').size().reset_index(name='total')
league_tendency = pd.merge(league_tendency, count_totals, on='count_state')
league_tendency['theory_prob'] = league_tendency['count'] / league_tendency['total']

theory_map = dict(zip(zip(league_tendency['count_state'], league_tendency['pitch_type_name']), league_tendency['theory_prob']))

# ---------------------------------------------------------
# 3. サプライズ度計算
# ---------------------------------------------------------
def calculate_surprise(row):
    key = (row['count_state'], row['pitch_type_name'])
    prob = theory_map.get(key, 0)
    return 1.0 - prob

df_play['surprise_score'] = df_play.apply(calculate_surprise, axis=1)

# 成功フラグ
df_play['is_success'] = ((df_play['is_swinging_strike'] == 1) | (df_play['is_called_strike'] == 1)).astype(int)

# スコア計算
df_play['strategic_score'] = df_play['surprise_score'] * df_play['is_success']
df_play['random_score'] = df_play['surprise_score'] * (1 - df_play['is_success'])

# ---------------------------------------------------------
# 4. ★ここが変更点：年度ごとに集計
# ---------------------------------------------------------
surprise_analysis = []

# 投手名 AND 年度 でグループ化
grouped = df_play.groupby(['pitcher_name', 'game_year'])

for (p_name, year), group in grouped:
    
    # 投球数が少なすぎる年（怪我など）は除外してもOK
    if len(group) < 10: 
        continue

    avg_strategic = group['strategic_score'].mean()
    avg_random = group['random_score'].mean()
    
    surprise_analysis.append({
        '投手名': p_name,
        '年度': year,
        'ラベル': f"{p_name}({year})", # グラフで点に名前を出す用
        '攻めるサプライズ': round(avg_strategic, 4),
        '無頓着サプライズ': round(avg_random, 4)
    })

df_surprise_yearly = pd.DataFrame(surprise_analysis)

# ---------------------------------------------------------
# 5. CSV保存
# ---------------------------------------------------------
csv_filename = 'pitcher_surprise_yearly.csv'
df_surprise_yearly.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print("✅ 計算完了！")
display(df_surprise_yearly.head(10)) # 最初の10行だけ表示
print(f"💾 '{csv_filename}' を保存しました。")
print("   → 1人の投手につき最大5つの点がプロットできるデータになっています！")

🧠 年度別サプライズ指数の計算を開始します...
✅ 計算完了！


,投手名,年度,ラベル,攻めるサプライズ,無頓着サプライズ
0,投手A_ドクターK,2021,投手A_ドクターK(2021),0.2705,0.4418
1,投手A_ドクターK,2022,投手A_ドクターK(2022),0.3091,0.3930
2,投手A_ドクターK,2023,投手A_ドクターK(2023),0.3026,0.4022
3,投手A_ドクターK,2024,投手A_ドクターK(2024),0.3068,0.4057
4,投手A_ドクターK,2025,投手A_ドクターK(2025),0.2950,0.4098
5,投手B_ドクターK,2021,投手B_ドクターK(2021),0.2703,0.4352
6,投手B_ドクターK,2022,投手B_ドクターK(2022),0.2875,0.4187
7,投手B_ドクターK,2023,投手B_ドクターK(2023),0.2870,0.4149
8,投手B_ドクターK,2024,投手B_ドクターK(2024),0.2627,0.4446
9,投手B_ドクターK,2025,投手B_ドクターK(2025),0.2640,0.4422


💾 'pitcher_surprise_yearly.csv' を保存しました。
   → 1人の投手につき最大5つの点がプロットできるデータになっています！
